##### Exemplo de dados

In [ ]:
# COD_UF;COD_MUN;COD_ESPECIE;LATITUDE;LONGITUDE;NV_GEO_COORD
# 11;1100015;1;-11.929621;-61.999058;1
# 11;1100015;7;-11.929365;-61.999312;1
# 11;1100015;6;-11.929400;-61.999499;1
# 11;1100015;6;-11.929364;-61.999695;1
# 11;1100015;1;-11.929381;-61.999262;1
# 11;1100015;1;-11.930439;-61.999046;1
# 11;1100015;7;-11.930571;-61.999006;1
# 11;1100015;1;-11.930311;-61.998982;1
# 11;1100015;1;-11.930717;-61.999009;1
# 11;1100015;1;-11.930282;-61.999566;1

#### AJUSTE DE DADOS

In [1]:
import csv
import os

In [80]:
arquivos_csv = [arq for arq in os.listdir('arquivos/') if arq.endswith('.csv')]
arquivos_csv

['11.csv', '12.csv', '13.csv', '14.csv', '15.csv']

In [71]:
dados = []

for i in arquivos_csv:
    dir_arquivo = os.path.join('arquivos/', i)
    with open(dir_arquivo, 'r') as arquivo:
        arquivo_csv = csv.reader(arquivo, delimiter=';')
        next(arquivo_csv)
        data = [linha for linha in arquivo_csv]
        dados.extend(data)

In [72]:
print(f'Total de registros: {len(dados)}')
dados[:5]

Total de registros: 7206549


[['11', '1100015', '1', '-11.929621', '-61.999058', '1'],
 ['11', '1100015', '7', '-11.929365', '-61.999312', '1'],
 ['11', '1100015', '6', '-11.929400', '-61.999499', '1'],
 ['11', '1100015', '6', '-11.929364', '-61.999695', '1'],
 ['11', '1100015', '1', '-11.929381', '-61.999262', '1']]

In [78]:
print(f'{dados[0][0].strip()} {dados[0][1].strip()} {dados[0][2].strip()} {dados[0][3].strip()} {dados[0][4].strip()} {dados[0][5].strip()}')

11 1100015 1 -11.929621 -61.999058 1


In [79]:
dados_aj = []

for i in range(len(dados)):
    aux = []
    var1 =   int(dados[i][0].strip())
    aux.append(var1)
    var2 =   int(dados[i][1].strip())
    aux.append(var2)
    var3 =   int(dados[i][2].strip())
    aux.append(var3)
    var4 = float(dados[i][3].strip())
    aux.append(var4)
    var5 = float(dados[i][4].strip())
    aux.append(var5)
    var6 =   int(dados[i][5].strip())
    aux.append(var6)

    dados_aj.append(aux)

#### PARTE DB POSTGRESQL

In [61]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

url = URL.create(
    drivername="postgresql",
    username="postgres",
    host="localhost",
    database="postgres",
    password="123"
)

engine = create_engine(url)

In [62]:
connection = engine.connect()

In [63]:
from sqlalchemy import Column, Integer, Float
from sqlalchemy.orm import declarative_base

In [64]:
Base = declarative_base()

In [65]:
class Localidade(Base):
    __tablename__ = 'ibge'

    id = Column(Integer(), primary_key=True, autoincrement=True)
    cod_uf = Column(Integer(), nullable=True)
    cod_mun = Column(Integer(), nullable=True)
    cod_especie = Column(Integer(), nullable=True)
    latitude = Column(Float(), nullable=True)
    longitude = Column(Float(), nullable=True)
    nv_geo_coord = Column(Integer(), nullable=True)


In [84]:
Localidade.__table__

Table('ibge', MetaData(), Column('id', Integer(), table=<ibge>, primary_key=True, nullable=False), Column('cod_uf', Integer(), table=<ibge>), Column('cod_mun', Integer(), table=<ibge>), Column('cod_especie', Integer(), table=<ibge>), Column('latitude', Float(), table=<ibge>), Column('longitude', Float(), table=<ibge>), Column('nv_geo_coord', Integer(), table=<ibge>), schema=None)

In [101]:
Base.metadata.create_all(engine)

#### INSERT DE DADOS NO POSTGRESQL

In [67]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [68]:
from sqlalchemy import insert

In [69]:
for i in range(len(dados_aj)):
    local1 = Localidade(cod_uf=dados_aj[i][0], cod_mun=dados_aj[i][1], cod_especie=dados_aj[i][2], latitude=dados_aj[i][3], longitude=dados_aj[i][4], nv_geo_coord=dados_aj[i][5])
    session.add(local1)
    session.commit()

In [88]:
quantidade = session.query(Localidade).count()
print(f"Total de registros registrados no DB: {quantidade}")

Total de registros registrados no DB: 7206549
